# Import libraries

In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import make_grid, save_image

from tensorboardX import SummaryWriter
from tqdm import tqdm
from copy import deepcopy


from utils import *
from models_final import *
from fid_score import *
from inception_score import *

In [2]:
#Necessary folders and files
!mkdir checkpoint
!mkdir generated_images
!pip install tensorboardX
!mkdir fid_stat
%cd fid_stat
!wget bioinf.jku.at/research/ttur/ttur_stats/fid_stats_cifar10_train.npz
%cd ..

A subdirectory or file checkpoint already exists.
A subdirectory or file generated_images already exists.


C:\Users\malin\TransGAN\fid_stat


A subdirectory or file fid_stat already exists.


C:\Users\malin\TransGAN


'wget' is not recognized as an internal or external command,
operable program or batch file.


# Hyperparameters for CIFAR-10 Dataset

In [2]:
# training hyperparameters given by code author

lr_gen = 0.0001 #Learning rate for generator
lr_dis = 0.0001 #Learning rate for discriminator
latent_dim = 1024 #Latent dimension
gener_batch_size = 32 #Batch size for generator
dis_batch_size = 32 #Batch size for discriminator
epoch = 300 #Number of epoch
weight_decay = 1e-3 #Weight decay
drop_rate = 0.5 #dropout
n_critic = 5 #
max_iter = 500000
img_name = "img_name"
lr_decay = True

# architecture details by authors
image_size = 32 #H,W size of image for discriminator
img_size = 32
initial_size = 8 #Initial size for generator
patch_size = 4 #Patch size for generated image
num_classes = 1 #Number of classes for discriminator 
output_dir = 'checkpoint' #saved model path
dim = 1024 #Embedding dimension 
optimizer = 'Adam' #Optimizer
loss = "wgangp_eps" #Loss function
phi = 1 #
beta1 = 0 # Adam Optimizer param
beta2 = 0.99 # Adam Optimizer param
diff_aug = "translation,cutout,color" #data augmentation

# Training

In [3]:
#FID
fid_stat = 'fid_stat/fid_stats_cifar10_train.npz'

writer=SummaryWriter()
writer_dict = {'writer':writer}
writer_dict["train_global_steps"]=0
writer_dict["valid_global_steps"]=0

In [4]:
def compute_gradient_penalty(D, real_samples, fake_samples, phi):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.Tensor(np.random.random((real_samples.size(0), 1, 1, 1))).to(real_samples.get_device())
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.ones([real_samples.shape[0], 1], requires_grad=False).to(real_samples.get_device())
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.contiguous().view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - phi) ** 2).mean()
    return gradient_penalty

In [5]:
def train(noise,generator, discriminator, optim_gen, optim_dis, train_loader,
        epoch, writer, schedulers, img_size=32, latent_dim = latent_dim,
        n_critic = n_critic,
        gener_batch_size=gener_batch_size, device="cuda:0"):

    writer = writer_dict['writer']
    gen_step = 0
    dis_step = 0
    gloss = 0.
    dloss = 0.

    generator.train()
    discriminator.train()

    for index, (img, _) in enumerate(train_loader):

        global_steps = writer_dict['train_global_steps']
        
        #Ground truth real image
        real_imgs = img.type(torch.cuda.FloatTensor).cuda(device, non_blocking=True)
        
        ###
        #Train Discriminator
        ###
        
        optim_dis.zero_grad()
        
        #Sample Noise for Generator
        noise = torch.cuda.FloatTensor(np.random.normal(0, 1, (img.shape[0], latent_dim))).cuda(device, non_blocking=True)
        
        #Generate fake image
        fake_imgs = generator(noise).detach()
        
        #Validate
        real_valid = discriminator(real_imgs)
        fake_valid = discriminator(fake_imgs)

        #Loss calculation - WGAN GP
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs, fake_imgs.detach(), phi)
        loss_dis = -torch.mean(real_valid) + torch.mean(fake_valid) + gradient_penalty * 10 / (phi ** 2)         

        loss_dis.backward()
        optim_dis.step()

        writer.add_scalar("loss_dis", loss_dis.item(), global_steps)
        
        optim_dis.zero_grad()
        
        dis_step += 1
        dloss += loss_dis.item()
        
        ###
        #Train Generator
        ###
        
        if global_steps % n_critic == 0:
            
            optim_gen.zero_grad()
            
            noise_gen = torch.cuda.FloatTensor(np.random.normal(0, 1, (gener_batch_size, latent_dim)))
            generated_imgs= generator(noise_gen)
            fake_valid = discriminator(generated_imgs)
            
            #Loss calculation
            loss_gen = -torch.mean(fake_valid)#.to(device)
            
            loss_gen.backward()
            optim_gen.step()
            
            writer.add_scalar("loss_gen", loss_gen.item(), global_steps)
            
            gen_step += 1
            gloss += loss_gen.item()
        
        #Log
        if gen_step and index % 100 == 0:
            sample_imgs = generated_imgs[:25]
            img_grid = make_grid(sample_imgs, nrow=5, normalize=True, scale_each=True)
            save_image(sample_imgs, f'generated_images/generated_img_{epoch}_{index % len(train_loader)}.jpg', 
                       nrow=5, normalize=True, scale_each=True)            
            tqdm.write("[Epoch %d] [Batch %d/%d] [D loss: %f] [G loss: %f]" %
                (epoch+1, index % len(train_loader), len(train_loader), loss_dis.item(), loss_gen.item()))
            
        writer_dict['train_global_steps'] = global_steps + 1
    
    #Learning rate scheduler
    if schedulers:
        gen_scheduler, dis_scheduler = schedulers
        gen_scheduler.step()
        g_lr = gen_scheduler.get_last_lr()
        dis_scheduler.step()
        d_lr = dis_scheduler.get_last_lr()
        writer.add_scalar('LR/d_lr', d_lr, global_steps)
        writer.add_scalar('LR/g_lr', g_lr, global_steps)
    
    return gloss/gen_step, dloss/dis_step

In [6]:
def validate(generator, writer_dict, fid_stat):

    writer = writer_dict['writer']
    global_steps = writer_dict['valid_global_steps']

    generator = generator.eval()
    fid_score = get_fid(fid_stat, generator, num_img=4096, val_batch_size=16, latent_dim=1024)
    print(f"FID score: {fid_score}")
    
    is_mean, is_std = get_is(generator, num_img=4096, val_batch_size=16, latent_dim=1024)
    print(f"Inception score: {is_mean}")

    writer.add_scalar('FID_score', fid_score, global_steps)
    writer.add_scalar('Inception_score', is_mean, global_steps)

    writer_dict['valid_global_steps'] = global_steps + 1
    return fid_score, is_mean, is_std

In [7]:
def Record_Train(epoch_record,dis_lr,gen_lr,dis_loss,gen_loss,fid_record,is_record):
    TrainingRecord = [epoch_record,dis_lr,gen_lr,dis_loss,gen_loss,fid_record,is_record]
    f = open("TrainingRecord.txt", "a")
    f.write('TrainingRecord\n')
    f.write('TrainingRecord = [epoch_record,dis_lr,gen_lr,dis_loss,gen_loss,fid_record,is_record]\n')
    f.write(str(TrainingRecord)+'\n')
    f.close()

# Config

In [8]:
#Assign device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [9]:
#Generator and Discriminator
generator= Generator(layer1=5, layer2=4, layer3=2, patch_size=8, dim=1024, heads=4, mlp_ratio=4, drop_rate=0.5)
generator.to(device)

discriminator = Discriminator(diff_aug = diff_aug, layer1=3, layer2=3, img_size=32, patch_size=4, input_channel=3, num_classes=1,
                 dim=384, heads=4, mlp_ratio=4, drop_rate=0.5)
discriminator.to(device)

generator.apply(inits_weight)
discriminator.apply(inits_weight)
print('Generator and Discriminator defined')

C:\Users\malin\TransGAN\utils.py:41: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight.data, 1.)


Generator and Discriminator defined


In [9]:
#Generator and Discriminator original
generator= Generator(depth1=5, depth2=4, depth3=3, initial_size=initial_size, dim=384, heads=4, mlp_ratio=4, drop_rate=0.5)#,device = device)
generator.to(device)

discriminator = Discriminator(diff_aug = diff_aug, image_size=32, patch_size=4, input_channel=3, num_classes=1,
                 dim=384, depth=8, heads=4, mlp_ratio=4,
                 drop_rate=0.5)
discriminator.to(device)


generator.apply(inits_weight)
discriminator.apply(inits_weight)
print('Generator and Discriminator defined')

C:\Users\malin\TransGAN\utils.py:41: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight.data, 1.)


Generator and Discriminator defined


In [10]:
#Optimizer
if optimizer == 'Adam':
    optim_gen = optim.Adam(filter(lambda p: p.requires_grad, generator.parameters()), lr=lr_gen, betas=(beta1, beta2))
    optim_dis = optim.Adam(filter(lambda p: p.requires_grad, discriminator.parameters()),lr=lr_dis, betas=(beta1, beta2))
    
elif optimizer == 'AdamW':
    optim_gen = optim.AdamW(filter(lambda p: p.requires_grad, generator.parameters()), lr=lr_gen, betas=(beta1, beta2), 
                            weight_decay=weight_decay)
    optim_dis = optim.AdamW(filter(lambda p: p.requires_grad, discriminator.parameters()),lr=lr_dis, betas=(beta1, beta2), 
                            weight_decay=weight_decay)


#gen_scheduler = LinearLrDecay(optim_gen, lr_gen, 0.0, 0, max_iter * n_critic)
#dis_scheduler = LinearLrDecay(optim_dis, lr_dis, 0.0, 0, max_iter * n_critic)

gen_scheduler = optim.lr_scheduler.CosineAnnealingLR(optim_gen, T_max=epoch)
dis_scheduler = optim.lr_scheduler.CosineAnnealingLR(optim_dis, T_max=epoch)

print("optimizer:",optimizer)

optimizer: Adam


In [11]:
#Load dataset
transform = transforms.Compose([transforms.Resize(size=(img_size, img_size)),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=32, shuffle=True)

Files already downloaded and verified


In [12]:
best = 1e4
best_epoch = 0

epoch_record= []
dis_lr = []
gen_lr = []
dis_loss = []
gen_loss = []
fid_record = []
is_record = [[],[]]

# Main

In [13]:
#Main training loop
for epoch in range(epoch):
    
    print(f'------- Epoch {epoch+1:3} -------')
    lr_schedulers = (gen_scheduler, dis_scheduler) if lr_decay else None
    
    epoch_record.append(epoch)
    dis_lr.append(optim_dis.param_groups[0]['lr'])
    gen_lr.append(optim_gen.param_groups[0]['lr'])
    
    gloss, dloss = train(noise, generator, discriminator, optim_gen, optim_dis, train_loader,
                        epoch, writer, lr_schedulers,img_size=32, latent_dim = latent_dim,
                        n_critic = n_critic,
                        gener_batch_size=gener_batch_size)
    
    dis_loss.append(dloss)
    gen_loss.append(gloss)
    
    checkpoint = {'epoch':epoch, 'best_fid':best}
    checkpoint['generator_state_dict'] = generator.state_dict()
    checkpoint['discriminator_state_dict'] = discriminator.state_dict()

    fid_score, is_mean, is_std = validate(generator, writer_dict, fid_stat)
    fid_record.append(fid_score)
    is_record[0].append(is_mean)
    is_record[1].append(is_std)

    if fid_score < best:
        save_checkpoint(checkpoint, is_best=True, output_dir=output_dir)
        print("Saved Latest Model!")
        best = fid_score
        best_epoch = epoch
    
    if epoch % 10 == 0:
        checkpoint = {'epoch':epoch, 'best_fid':best}
        checkpoint['generator_state_dict'] = generator.state_dict()
        checkpoint['discriminator_state_dict'] = discriminator.state_dict()
        save_checkpoint(checkpoint,is_best=(fid_score<best), output_dir=output_dir)
        Record_Train(epoch_record,dis_lr,gen_lr,dis_loss,gen_loss,fid_record,is_record)
    
    print('----- Epoch Summary -----')
    print(f'Dis Loss: {dloss:.6f} - Gen Loss: {gloss:.6f} - Dis LR: {dis_lr[len(dis_lr)-1]:.6} - Gen LR: {gen_lr[len(gen_lr)-1]:.6}')
    print(f'FID score: {fid_score} - Inception score: {is_mean} || @ epoch {epoch+1}.')
    print(f'Best FID score: {best} @ epoch {best_epoch+1}')
    print('-----      End      -----')
    print('')
    
    #torch.cuda.empty_cache()

checkpoint = {'epoch':epoch, 'best_fid':best}
checkpoint['generator_state_dict'] = generator.state_dict()
checkpoint['discriminator_state_dict'] = discriminator.state_dict()
fid_score,_,_ = validate(generator, writer_dict, fid_stat) ####CHECK AGAIN
save_checkpoint(checkpoint,is_best=(fid_score<best), output_dir=output_dir)
Record_Train(epoch_record,dis_lr,gen_lr,dis_loss,gen_loss,fid_record,is_record)

------- Epoch   1 -------


C:\Users\malin\anaconda3\envs\AI6103\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[Epoch 1] [Batch 0/1563] [D loss: 3.556046] [G loss: -0.744356]
[Epoch 1] [Batch 100/1563] [D loss: -29.021587] [G loss: 8.220548]
[Epoch 1] [Batch 200/1563] [D loss: -35.306534] [G loss: 15.706522]
[Epoch 1] [Batch 300/1563] [D loss: -20.103798] [G loss: 4.572022]
[Epoch 1] [Batch 400/1563] [D loss: -19.777868] [G loss: 12.333179]
[Epoch 1] [Batch 500/1563] [D loss: -5.337823] [G loss: 7.443689]
[Epoch 1] [Batch 600/1563] [D loss: -7.689364] [G loss: 5.157987]
[Epoch 1] [Batch 700/1563] [D loss: -14.346274] [G loss: 9.502428]
[Epoch 1] [Batch 800/1563] [D loss: -6.218648] [G loss: 7.400649]
[Epoch 1] [Batch 900/1563] [D loss: -8.657193] [G loss: 8.524136]
[Epoch 1] [Batch 1000/1563] [D loss: -7.707193] [G loss: 9.040615]
[Epoch 1] [Batch 1100/1563] [D loss: -4.985080] [G loss: 8.176661]
[Epoch 1] [Batch 1200/1563] [D loss: -3.068681] [G loss: 5.416013]
[Epoch 1] [Batch 1300/1563] [D loss: -10.500183] [G loss: 6.058139]
[Epoch 1] [Batch 1400/1563] [D loss: -1.637666] [G loss: 2.457803]

sample images: 100%|██████████| 256/256 [00:05<00:00, 45.30it/s]
C:\Users\malin\anaconda3\envs\AI6103\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\malin\anaconda3\envs\AI6103\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\malin\anaconda3\envs\AI6103\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


100%|██████████| 81/81 [00:04<00:00, 17.46it/s]


...

sample images: 100%|██████████| 256/256 [00:05<00:00, 48.03it/s]


100%|██████████| 81/81 [00:04<00:00, 19.79it/s]


FID score: 50.919097900390625


sample images: 100%|██████████| 256/256 [00:05<00:00, 47.96it/s]


Inception score: 5.387210447091289
